# Obsidian to LaTeX Converter - Jupyter Interface

This notebook provides an interactive interface to convert Obsidian markdown files to LaTeX sections.

In [1]:
# Import necessary libraries
import os
import sys
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import traceback

# Add the parent directory to the path to import the obsidian_to_latex package
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import the converter
from obsidian_to_latex.converter import ObsidianLatexSectionConverter

## File Selection Widgets

Select your Obsidian markdown file and output options.

In [2]:
# Get the current directory
current_dir = os.getcwd()

# Function to list available markdown files
def list_markdown_files(directory=current_dir):
    md_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.md'):
                md_files.append(os.path.join(root, file))
    return md_files

# Create widgets
input_file_dropdown = widgets.Dropdown(
    options=list_markdown_files(),
    description='Input File:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

output_file = widgets.Text(
    value='output.tex',
    description='Output File:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

figures_dir = widgets.Text(
    value='figures',
    description='Figures Directory:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

level_adjust = widgets.IntSlider(
    value=0,
    min=-2,
    max=2,
    step=1,
    description='Header Level Adjustment:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# Add file overwrite mode dropdown
overwrite_mode = widgets.Dropdown(
    options=[
        ('Overwrite existing file', 'overwrite'),
        ('Create backup before overwriting', 'backup'),
        ('Skip if file exists', 'skip')
    ],
    value='overwrite',
    description='If file exists:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

verbose_checkbox = widgets.Checkbox(
    value=False,
    description='Verbose Mode',
    style={'description_width': 'initial'}
)

# Display widgets
display(input_file_dropdown, output_file, figures_dir, level_adjust, overwrite_mode, verbose_checkbox)

Dropdown(description='Input File:', layout=Layout(width='80%'), options=(), style=DescriptionStyle(description…

Text(value='output.tex', description='Output File:', layout=Layout(width='80%'), style=TextStyle(description_w…

Text(value='figures', description='Figures Directory:', layout=Layout(width='80%'), style=TextStyle(descriptio…

IntSlider(value=0, description='Header Level Adjustment:', layout=Layout(width='50%'), max=2, min=-2, style=Sl…

Dropdown(description='If file exists:', layout=Layout(width='80%'), options=(('Overwrite existing file', 'over…

Checkbox(value=False, description='Verbose Mode', style=CheckboxStyle(description_width='initial'))

## Convert Button and Output

In [3]:
# Create output widgets
output_area = widgets.Output()
preview_area = widgets.Output()

# Create conversion function
def convert_file(b):
    # Clear previous output
    with output_area:
        clear_output()
    with preview_area:
        clear_output()
    
    # Get values from widgets
    input_file = input_file_dropdown.value
    output_file_path = output_file.value
    figures_directory = figures_dir.value
    level_adjustment = level_adjust.value
    file_overwrite_mode = overwrite_mode.value
    verbose = verbose_checkbox.value
    
    # Create log file path (in same directory as output file)
    log_file_path = os.path.join(
        os.path.dirname(output_file_path) if os.path.dirname(output_file_path) else ".",
        "obsidian2latex_debug.log"
    )
    
    # Create converter with log file
    converter = ObsidianLatexSectionConverter(
        input_file=input_file,
        output_file=output_file_path,
        figures_dir=figures_directory,
        verbose=verbose,
        log_file=log_file_path
    )
    
    # Perform conversion
    with output_area:
        try:
            print(f"Converting {input_file} to {output_file_path}...")
            print(f"File overwrite mode: {file_overwrite_mode}")
            print(f"Detailed log will be saved to: {log_file_path}")
            
            latex_content = converter.convert(level_adjustment=level_adjustment)
            
            if latex_content:
                # Pass the overwrite mode to the save method
                success = converter.save(latex_content, overwrite_mode=file_overwrite_mode)
                
                if success:
                    if file_overwrite_mode == 'skip' and os.path.exists(output_file_path):
                        print(f"Existing file was not modified: {os.path.abspath(output_file_path)}")
                    else:
                        converter.process_images()
                        print(f"Conversion completed successfully!")
                        print(f"Output saved to: {os.path.abspath(output_file_path)}")
                    
                    # Show LaTeX Preview
                    with preview_area:
                        print("LaTeX Preview (first 500 characters):")
                        print("\n" + "=" * 80 + "\n")
                        preview_content = latex_content[:500]
                        if len(latex_content) > 500:
                            preview_content += "\n... (content truncated)"
                        print(preview_content)
                else:
                    print("Failed to save output file.")
                    print(f"See log for details: {os.path.abspath(log_file_path)}")
            else:
                print("Conversion failed.")
                print(f"See log for details: {os.path.abspath(log_file_path)}")
        except Exception as e:
            print(f"An error occurred: {e}")
            print(traceback.format_exc())
            print(f"\nDetailed error information saved to: {os.path.abspath(log_file_path)}")

# Function to view the log file
def view_log_file(b):
    log_file_path = os.path.join(
        os.path.dirname(output_file.value) if os.path.dirname(output_file.value) else ".",
        "obsidian2latex_debug.log"
    )
    
    with output_area:
        clear_output()
        try:
            if os.path.exists(log_file_path):
                with open(log_file_path, 'r') as f:
                    log_content = f.read()
                print(f"Log file content ({log_file_path}):")
                print("=" * 80)
                print(log_content)
            else:
                print(f"Log file not found: {log_file_path}")
        except Exception as e:
            print(f"Error reading log file: {e}")

# Create buttons
convert_button = widgets.Button(
    description='Convert to LaTeX',
    button_style='primary',
    tooltip='Convert Obsidian markdown to LaTeX'
)

view_log_button = widgets.Button(
    description='View Log File',
    button_style='info',
    tooltip='View the detailed log file'
)

convert_button.on_click(convert_file)
view_log_button.on_click(view_log_file)

# Display buttons in a row
button_row = widgets.HBox([convert_button, view_log_button])
display(button_row)
display(widgets.HTML("<h3>Log Output:</h3>"))
display(output_area)
display(widgets.HTML("<h3>LaTeX Preview:</h3>"))
display(preview_area)

HTML(value='<h3>Log Output:</h3>')

Output()

HTML(value='<h3>LaTeX Preview:</h3>')

Output()

## Tips for Using the Converter

1. **Header Levels**: Use the header level adjustment to fit your LaTeX document structure. For example, if your Obsidian file uses `#` headers that should be `\subsection` in LaTeX, set the adjustment to `1`.

2. **Images**: The converter will try to find images in the following locations:
   - `attachments` folder in the same directory as the markdown file
   - `assets` folder in the same directory
   - `images` folder in the same directory
   - `attachments` folder in the parent directory

3. **Math Expressions**: The converter preserves math expressions in the document. It handles both inline math (`$...$`) and display math (`$$...$$`).

4. **Including in LaTeX Documents**: To include the generated section in your LaTeX document, use `\input{path/to/output.tex}` in your main LaTeX file.

5. **File Handling Options**:
   - **Overwrite existing file**: Replaces any existing file with the same name
   - **Create backup before overwriting**: Creates a timestamped backup of the existing file before replacing it
   - **Skip if file exists**: Leaves the existing file untouched if it already exists